# Compare before & after CB

In [1]:
import os, sys, re, io, math
import matplotlib, shap, xgboost
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import *
from math import isnan, nan
from matplotlib.widgets import Slider
from glob import glob
from ipywidgets import *
from datetime import datetime, timedelta
from dateutil.tz import tzlocal
import ipywidgets as widgets
from IPython.display import *
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.options.display.width = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100000
pd.options.display.float_format = '{0:f}'.format

# Disable scrolling for all output widgets
style = """
    <style>
       .jupyter-widgets-output-area .output_scroll {
            height: unset !important;
            border-radius: unset !important;
            -webkit-box-shadow: unset !important;
            box-shadow: unset !important;
        }
        .jupyter-widgets-output-area  {
            height: auto !important;
        }
    </style>
    """
display(HTML(style))

from pandas_serializer import *
from compare_CB import *

In [2]:
os.CB_start_date = pd.Timestamp('2020-04-07', tz='tzlocal()')
os.CB_boundary_gap = pd.to_timedelta('7D')
all_data = pandas_load('5.decrypted/izedAa85XXrDS85XlwrOsIDU/all-data.pson.gz')

# Compare stats

In [3]:
@interact(show=widgets.ToggleButton(value=False,description='Show Comparison'))
def show_compare(show):
    if not show:
        return clear_output()
    display(get_compare(all_data))

interactive(children=(ToggleButton(value=False, description='Show Comparison'), Output()), _dom_classes=('widg…

# Plot Shap

In [4]:
@interact(show=widgets.ToggleButton(value=False,description='Show Shap Plot'))
def show_shap(show):
    if not show:
        return clear_output()
    f, shap_values, X = get_shap(all_data)
    # f.savefig("/summary_plot1.png", bbox_inches='tight', dpi=600)

interactive(children=(ToggleButton(value=False, description='Show Shap Plot'), Output()), _dom_classes=('widge…

# For Testing

In [5]:
for ii,(name, df) in enumerate(all_data.items()):
    if ii!=2 or True:
        continue
    df1 = df[[col for col in df.columns if (col.startswith('tapsLog') and 'dur' in col)]]
    display(name)
    display(df1.max())
